# Data Frames 

¿Cuales son los beneficios de ocupar un dataframe?
- Permite procesar como una tabla de bases de datos los DF.
- Poseen estructura y pueden ser creados como lo Df.
- Una optimización superior debido al optimizador de consultas Catalyst y el 
  motor de ejecución Tungsten.

In [1]:
import findspark
findspark.init()

In [25]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType,FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [27]:
sc = SparkContext.getOrCreate() 

#### Creación de un Data Frame

In [28]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

Para ver un número determinado de lineas con el comando linux lo hacemos mediante el operador !head -n 5 dónde el número nos dice cuantas lineas se van a mostrar.

In [29]:
!ls resources/files/

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [30]:
!head -n 5  resources/files/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina


## Carga de un archivo desde cero
si nosotros deseamos cargar un archivo desde cero con spark el primer paso será crear nuestro esquema el cual hace uso de un StructType y un StructField.

Para crear nuestro dataframe a partir de un esquema lo hacemos de la siguiente manera:

1. Pasamos el esquema
2. Decimos que la primer fila es un header.
3. Pasamos la ruta de nuestro archivo.

In [33]:
path = "resources/files/"
juegoSchema = StructType([
    StructField("juego_id", IntegerType(), False),
    StructField("anio", StringType(), False),
    StructField("Ciudad",StringType(), False)
])

juegoDF = sqlContext.read.schema(juegoSchema) \
    .option("header", "true") \
    . csv(path + "juegos.csv")

In [37]:
juegoDF.show(5)

+--------+-----------+------+
|juego_id|       anio|Ciudad|
+--------+-----------+------+
|       1|1896 Verano|  1896|
|       2|1900 Verano|  1900|
|       3|1904 Verano|  1904|
|       4|1906 Verano|  1906|
|       5|1908 Verano|  1908|
+--------+-----------+------+
only showing top 5 rows



### Creación de un Data Frame apartir de un RDD


In [79]:
deportistaOlimpicoRDD = sc.textFile(path + "deportista.csv") \
    .map(lambda l: l.split(","))

In [80]:
deportistaOlimpicoRDD.take(4)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273']]

In [81]:
'''
    FUNCION QUE ELIMINA EL ENCABEZADO DE NUESTRO RDD
'''
def eliminaEncabezado(indice, iterador):
    return iter(list(iterador)[1:])


Para aplicar una función a un RDD lo hacemos mediante la función mapPartitionsWithIndex()
recibirá una función como parámetro y esta asu vez le pasará 2 :
- indice 
- Valor de toda la columna

In [82]:
deportistaOlimpicoRDD.take(7)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705'],
 ['6', 'Per Knut Aaland', '1', '31', '188', '75', '1096']]

In [83]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .mapPartitionsWithIndex(eliminaEncabezado)

In [84]:
deportistaOlimpicoRDD.take(4)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

### Asignación de tipo de dato al RDD deportista olimipico

In [85]:
deportistaOlimpicoRDD =  deportistaOlimpicoRDD.map(lambda l : [
     int(l[0]), # Deportista id 
     l[1], # nombre 
     int(l[2]), # genero
     int(l[3]), #edad
     int(l[4]), #altura
     float(l[5]), #peso
     int(l[6]),# equipo_id 
])

In [86]:
deportistaOlimpicoRDD.take(3)

[[1, 'A Dijiang', 1, 24, 180, 80.0, 199],
 [2, 'A Lamusi', 1, 23, 170, 60.0, 199],
 [3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273]]

#### Creación del esquema  para el RDD deportistaOlimpicoRDD

In [88]:
schema = StructType([
        StructField("deportista_id", IntegerType(), False),
        StructField("nombre", StringType(), False),
        StructField("genero", IntegerType(),True),
        StructField("edad",  IntegerType(), True),
        StructField("altura", IntegerType(),True),
        StructField("peso",  FloatType(), True),
        StructField("equipo_id",IntegerType(), True)
])

#### Creación del data frame 

Para crear el data frame lo haremos mediante el sqlcontext el cual le pasaremos como parámetros
1. RDD a transformar
2. Schema que vamos a utilizar



In [90]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)